# Plotting

Access optimization results and create visualizations.

This notebook covers:

- Loading saved FlowSystems from NetCDF files
- Accessing data (flow rates, sizes, effects, charge states)
- Time series plots (balance, flows, storage)
- Aggregated plots (sizes, effects, duration curves)
- Heatmaps with time reshaping
- Sankey diagrams
- Topology visualization
- Color customization and export

## Setup

In [ ]:
from pathlib import Path

import flixopt as fx

fx.CONFIG.notebook()

## Generate Example Data

First, run the script that generates three example FlowSystems with solutions:

In [ ]:
# Run the generation script (only needed once, or to regenerate)
!python data/generate_example_systems.py > /dev/null 2>&1

## 1. Loading Saved FlowSystems

FlowSystems can be saved to and loaded from NetCDF files, preserving the full structure and solution:

In [ ]:
DATA_DIR = Path('data')

# Load the three example systems
simple = fx.FlowSystem.from_netcdf(DATA_DIR / 'simple_system.nc4')
complex_sys = fx.FlowSystem.from_netcdf(DATA_DIR / 'complex_system.nc4')
multiperiod = fx.FlowSystem.from_netcdf(DATA_DIR / 'multiperiod_system.nc4')

print('Loaded systems:')
print(f'  simple:      {len(simple.components)} components, {len(simple.buses)} buses')
print(f'  complex_sys: {len(complex_sys.components)} components, {len(complex_sys.buses)} buses')
print(f'  multiperiod: {len(multiperiod.components)} components, dims={dict(multiperiod.solution.sizes)}')

## 2. Quick Overview: Balance Plot

Let's start with the most common visualization - a balance plot showing energy flows:

In [ ]:
# Balance plot for the Heat bus - shows all inflows and outflows
simple.statistics.plot.balance('Heat')

### Accessing Plot Data

Every plot returns a `PlotResult` with both the figure and underlying data. Use `.data.to_dataframe()` to get a pandas DataFrame:

In [ ]:
# Get plot result and access the underlying data
result = simple.statistics.plot.balance('Heat', show=False)

# Convert to DataFrame for easy viewing/export
df = result.data.to_dataframe()
df.head(10)

### Energy Totals

Get total energy by flow using `flow_hours`:

In [ ]:
import pandas as pd

# Total energy per flow
totals = {var: float(simple.statistics.flow_hours[var].sum()) for var in simple.statistics.flow_hours.data_vars}

pd.Series(totals, name='Energy [kWh]').to_frame().T

## 3. Time Series Plots

### 3.1 Balance Plot

Shows inflows (positive) and outflows (negative) for a bus or component:

In [ ]:
# Component balance (all flows of a component)
simple.statistics.plot.balance('ThermalStorage')

### 3.2 Carrier Balance

Shows all flows of a specific carrier across the entire system:

In [ ]:
complex_sys.statistics.plot.carrier_balance('heat')

In [ ]:
complex_sys.statistics.plot.carrier_balance('electricity')

### 3.3 Flow Rates

Plot multiple flow rates together:

In [ ]:
# All flows
simple.statistics.plot.flows()

In [ ]:
# Flows filtered by component
simple.statistics.plot.flows(component='Boiler')

### 3.4 Storage Plot

Combined view of storage charge state and flows:

In [ ]:
simple.statistics.plot.storage('ThermalStorage')

### 3.5 Charge States Plot

Plot charge state time series directly:

In [ ]:
simple.statistics.plot.charge_states('ThermalStorage')

## 4. Aggregated Plots

### 4.1 Sizes Plot

Bar chart of component/flow sizes:

In [ ]:
multiperiod.statistics.plot.sizes()

### 4.2 Effects Plot

Bar chart of effect totals by component:

In [ ]:
simple.statistics.plot.effects(effect='costs')

In [ ]:
# Multi-effect system: compare costs and CO2
complex_sys.statistics.plot.effects(effect='costs')

In [ ]:
complex_sys.statistics.plot.effects(effect='CO2')

### 4.3 Duration Curve

Shows how often each power level is reached:

In [ ]:
simple.statistics.plot.duration_curve('Boiler(Heat)')

In [ ]:
# Multiple variables
complex_sys.statistics.plot.duration_curve(['CHP(Heat)', 'HeatPump(Heat)', 'BackupBoiler(Heat)'])

## 5. Heatmaps

Heatmaps reshape time series into 2D grids (e.g., hour-of-day vs day):

In [ ]:
# Auto-reshape based on data frequency
simple.statistics.plot.heatmap('Boiler(Heat)')

In [ ]:
# Storage charge state heatmap
simple.statistics.plot.heatmap('ThermalStorage')

In [ ]:
# Custom colorscale
simple.statistics.plot.heatmap('Office(Heat)', color_continuous_scale='Blues', title='Heat Demand Pattern')

## 6. Sankey Diagrams

Sankey diagrams visualize energy flows through the system.

### 6.1 Flow Sankey

Total energy flows:

In [ ]:
simple.statistics.plot.sankey.flows()

In [ ]:
# Complex system with multiple carriers
complex_sys.statistics.plot.sankey.flows()

### 6.2 Sizes Sankey

Capacity/size allocation:

In [ ]:
multiperiod.statistics.plot.sankey.sizes()

### 6.3 Peak Flow Sankey

Maximum flow rates (peak power):

In [ ]:
simple.statistics.plot.sankey.peak_flow()

### 6.4 Effects Sankey

Cost/emission allocation:

In [ ]:
simple.statistics.plot.sankey.effects(select={'effect': 'costs'})

In [ ]:
# CO2 allocation in complex system
complex_sys.statistics.plot.sankey.effects(select={'effect': 'CO2'})

### 6.5 Filtering with `select`

Filter Sankey to specific buses or carriers:

In [ ]:
# Only heat flows
complex_sys.statistics.plot.sankey.flows(select={'bus': 'Heat'})

## 7. Topology Visualization

Visualize the system structure (no solution data required).

### 7.1 Topology Plot

Sankey-style network diagram:

In [ ]:
simple.topology.plot()

In [ ]:
complex_sys.topology.plot(title='Complex System Topology')

### 7.2 Topology Info

Get node and edge information programmatically:

In [ ]:
nodes, edges = simple.topology.infos()

print('Nodes:')
for label, info in nodes.items():
    print(f'  {label}: {info["class"]}')

print('\nEdges (flows):')
for label, info in edges.items():
    print(f'  {info["start"]} -> {info["end"]}: {label}')

## 8. Multi-Period/Scenario Data

Working with multi-dimensional results:

In [ ]:
print('Multiperiod system dimensions:')
print(f'  Periods: {list(multiperiod.periods)}')
print(f'  Scenarios: {list(multiperiod.scenarios)}')
print(f'  Solution dims: {dict(multiperiod.solution.sizes)}')

In [ ]:
# Balance plot with faceting by scenario
multiperiod.statistics.plot.balance('Heat')

In [ ]:
# Filter to specific scenario/period
multiperiod.statistics.plot.balance('Heat', select={'scenario': 'high_demand', 'period': 2024})

In [ ]:
# Sankey aggregates across all dimensions by default
multiperiod.statistics.plot.sankey.flows()

## 9. Color Customization

Colors can be customized in multiple ways:

In [ ]:
# Using a colorscale name
simple.statistics.plot.balance('Heat', colors='Set2')

In [ ]:
# Using a list of colors
simple.statistics.plot.balance('Heat', colors=['#e41a1c', '#377eb8', '#4daf4a', '#984ea3'])

In [ ]:
# Using a dictionary for specific labels
simple.statistics.plot.balance(
    'Heat',
    colors={
        'Boiler(Heat)': 'orangered',
        'ThermalStorage(Charge)': 'steelblue',
        'ThermalStorage(Discharge)': 'lightblue',
        'Office(Heat)': 'forestgreen',
    },
)

## 10. Exporting Results

Plots return a `PlotResult` with data and figure that can be exported:

In [ ]:
# Get plot result
result = simple.statistics.plot.balance('Heat')

print('PlotResult contains:')
print(f'  data: {type(result.data).__name__} with vars {list(result.data.data_vars)}')
print(f'  figure: {type(result.figure).__name__}')

In [ ]:
# Export data to pandas DataFrame
df = result.data.to_dataframe()
df.head()

In [ ]:
# Export figure to HTML (interactive)
# result.figure.write_html('balance_plot.html')

# Export figure to image
# result.figure.write_image('balance_plot.png', scale=2)

## Summary

### Data Access

| Property | Description |
|----------|-------------|
| `statistics.flow_rates` | Time series of flow rates (power) |
| `statistics.flow_hours` | Energy values (rate × duration) |
| `statistics.sizes` | Component/flow capacities |
| `statistics.charge_states` | Storage charge levels |
| `statistics.temporal_effects` | Effects per timestep |
| `statistics.periodic_effects` | Effects per period |
| `statistics.total_effects` | Aggregated effect totals |
| `topology.carrier_colors` | Cached carrier color mapping |
| `topology.component_colors` | Cached component color mapping |
| `topology.bus_colors` | Cached bus color mapping |

### Plot Methods

| Method | Description |
|--------|-------------|
| `plot.balance(node)` | Stacked bar of in/outflows |
| `plot.carrier_balance(carrier)` | Balance for all flows of a carrier |
| `plot.flows(variables)` | Time series line/area plot |
| `plot.storage(component)` | Combined charge state and flows |
| `plot.charge_states(component)` | Charge state time series |
| `plot.sizes()` | Bar chart of sizes |
| `plot.effects(effect)` | Bar chart of effect contributions |
| `plot.duration_curve(variables)` | Sorted duration curve |
| `plot.heatmap(variable)` | 2D time-reshaped heatmap |
| `plot.sankey.flows()` | Energy flow Sankey |
| `plot.sankey.sizes()` | Capacity Sankey |
| `plot.sankey.peak_flow()` | Peak power Sankey |
| `plot.sankey.effects(effect)` | Effect allocation Sankey |
| `topology.plot()` | System structure diagram |